*for more information, checkout* [Github](https://github.com/Crazytoph/2Dcorrelation "my github page").

# CD Measurements - Data Analysis

## *setting up the Notebook*

In [1]:
# Enabling the `widget` backend.
# This requires jupyter-matplotlib a.k.a. ipympl.
# ipympl can be install via pip or conda.
%matplotlib widget
        
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import Output
import matplotlib
from scipy import integrate
import os

In [2]:
import analise as ana 
import cdata 
import hotznplots as plot

<div class="alert alert-block alert-info">
<b>Info:</b> Print always whole DataFrames
</div>

In [3]:
# Default value of display.max_rows is 10 i.e. at max 10 rows will be printed.
# Set it None to display all rows in the dataframe
pd.set_option('display.max_rows', None)

<div class="alert alert-block alert-info">
<b>Info:</b> Get the data
</div>

In [4]:
# Path for Daniel:
#path = "F:\\HZDR\\CD_data"

# My Path:
path = "C:\\Users\\crazy\\Mega\\Uni\\Masterarbeit\\Projekt\\Data\\CD_data"
datalist = os.listdir(path)
print(datalist)

['Control_0M', 'Gdm2SO4_0.5M', 'Gdm2SO4_2M', 'Gdm2SO4_4M', 'GdmCl_0.5M', 'GdmCl_2M', 'GdmCl_2M_24h', 'GdmSCN_0.5M', 'GdmSCN_2M', 'Urea_2M']


In [5]:
# all data in one dictionary with foldernames as names
data_all = {}
for i in range(len(datalist)):
    data_all[datalist[i]] = cdata.CData(os.path.join(path, datalist[i]))

In [6]:
# options to acces data
print(data_all['Control_0M'])
# multiple options
print(list(map(data_all.get, ['Control_0M', 'Gdm2SO4_4M'])))
print(tuple(data_all.values())[0:4])

[<cdata.CData object at 0x000001C9A15E47F0>, <cdata.CData object at 0x000001C9A15B16A0>]
(<cdata.CData object at 0x000001C9A15E47F0>, <cdata.CData object at 0x000001C99DD47130>, <cdata.CData object at 0x000001C99DD476D0>, <cdata.CData object at 0x000001C9A15B16A0>)


## Colormaps

In [7]:
plot.heatmap(*tuple(data_all.values()), subtitle=tuple(data_all.keys()), title="Cd-Values")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Function Plots 

In [8]:
data=[]
for key in data_all.keys():
    data.append([data_all[key]])

plot.mult_func([230, 247, 275, 300], *data, subtitle=tuple(data_all.keys()), title="Cd-Values")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Correlation Analyis

In [9]:
plot.mult_func([247, "fit"], [ana.sigmoid_fit(ana.normalize(data_all["Control_0M"].t_df))],  title="Sigmoid-Fit", subtitle=[""], marker=['x', ''])

[ 0.18677103 61.81964853] [[ 0.00038838 -0.00105735]
 [-0.00105735  0.28747641]]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …